# Task A: Dataset Exploration & Analysis

## Objective
- Load and inspect the maritime fleet dataset
- Understand schema, units, and data quality
- Document assumptions for ML model
- Identify features and target variable


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

print("Environment ready!")


## 1. Load Dataset

Dataset exploration will be completed in Commit 2
